Долгодворова Маша, ДЗ№1

# Датасет с ключевыми словами

- Были взяты данные из репозитория к статье SemEval-2010 Task 5: Automatic Keyphrase Extraction from Scientific Articles https://github.com/snkim/AutomaticKeyphraseExtraction
- Из имеющихся датасетов с выделенными ключевыми словами выбран следующий: https://github.com/snkim/AutomaticKeyphraseExtraction/blob/master/Nguyen2007.zip - внутри которого были выбраны 3 текста 
- Ключевые фразы хранятся в отдельном файле .key
- В результате получился csv файл вида text_id | text | keywords | my_keywords

# 1. Подготовка корпуса

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import RAKE
from summa import keywords
from collections import Counter
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import MultiLabelBinarizer

import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation, ascii_lowercase, digits
mystem = Mystem()

1. Считываем данные

In [3]:
df = pd.read_csv('nlp_keys.csv')
df.columns = ['text_id', 'text', 'keywords', 'my_keywords']
df.head(3)

,text_id,text,keywords,my_keywords
0,1,Entropy and Self-Organization in Multi-Agent S...,"self-organization, pheromones, entropy","multi-agent systems, self-organization, pherom..."
1,2,Diagnosis of TCP Overlay Connection Failures u...,"bayesian networks, fault diagnosis, passive di...","codeen, bayesian networks, fault diagnosis, pa..."
2,3,Efficient Multi-way Text Categorization via Ge...,"multi-class text categorization, gsvd, discrim...","lsi, multi-class text categorization, gsvd, di..."


2. Лемматизация

In [4]:
def preprocess_text(text):
    """
    На вход подается считанный файл
    Приведение к одному регистру, удаление пунктуации и стоп-слов, лемматизация
    Возвращает леммы 
    """
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if not set(token).intersection(digits)
              and token != ', ' and token != " " and token not in punctuation]
    return tokens

In [5]:
# некоторые вещи легче убрать так, чем в препроцессинге
df['text'] = df.text.replace({'\n':''}, regex=True).replace({'[()]':''}, regex=True)

In [6]:
texts = df['text'].tolist()

In [7]:
l = 0
corpus = []
for t in texts:
    res = preprocess_text(t)
    l += len(res)
    corpus.append(' '.join(res))

In [8]:
print(f" Получившийся объем корпуса: {l} токенов ")

 Получившийся объем корпуса: 16095 токенов 


# Разметка ключевых слов и анализ 

При выделении ключевых слов (представленных в my_keywords) я была полностью согласна с выделенными ключевыми словами, однако для каждого текста мне показалось стоящим добавить несколько ключевых слов - в результате мои ключевые слова это объединение с исходной разметкой
Добавлять слишком много ключевых слов мне кажется не совсем верным - такой подход не помогает в решении задач NLP Например, в задаче извлечении информации:
- [*Роль общей и специфической лексики при извлечении информации из текста на примере анализа события «ввод новых технологий»* Юлия Сергеевна Акинина; Анастасия Александровна Бонч-Осмоловская; Илья Олегович Кузнецов; Виктор Петрович Клинцов; Светлана Юрьевна Толдова] https://lib.nsu.ru/xmlui/handle/nsu/257

In [9]:
my_keys = df['my_keywords'].tolist() # все ключевые слова (мои + изначальные)

In [10]:
origin_keys = df['keywords'].tolist()

1. Анализ ключевых слов

In [11]:
s = 0
for k in my_keys:
    all_k = k.split(', ')
    origin_k = origin_keys[s].split(', ')
    intersection = list(set(all_k) & set(origin_k))
    s+=1
    print(f"Изначально в {s} тексте выделено ключевых слов: {len(origin_k)} ")
    print(f"Добавлено при ручной разметке ключевых слов: {len(all_k) - len(origin_k)}")
    print(f"Всего ключевых слов: {len(all_k)}")
    print('- - - - - - - - - - - - - - - - - - - - - - - - - -')

Изначально в 1 тексте выделено ключевых слов: 3 
Добавлено при ручной разметке ключевых слов: 2
Всего ключевых слов: 5
- - - - - - - - - - - - - - - - - - - - - - - - - -
Изначально в 2 тексте выделено ключевых слов: 5 
Добавлено при ручной разметке ключевых слов: 2
Всего ключевых слов: 7
- - - - - - - - - - - - - - - - - - - - - - - - - -
Изначально в 3 тексте выделено ключевых слов: 3 
Добавлено при ручной разметке ключевых слов: 1
Всего ключевых слов: 4
- - - - - - - - - - - - - - - - - - - - - - - - - -


# 3 метода извлечения ключевых слов 

Я выбрала RAKE, Tf-Idf, TextRank

1. Tf-Idf

In [12]:
tf_vectorizer = TfidfVectorizer()
tf_key = []

tf_keywords = tf_vectorizer.fit_transform(corpus)
words = np.array(tf_vectorizer.get_feature_names())
arg = np.argsort(tf_keywords.toarray())

for i in arg:
    index = i[::-1]
    tf_key.append(words[index][:10].tolist())

2. Rake

In [13]:
stop_w = stopwords.words('english')
rake = RAKE.Rake(stop_w)
rake_key = []
for i in corpus:
    k = rake.run(i, maxWords=6, minFrequency=2)
    rake_key.append([i[0] for i in k])

3. TextRank

In [14]:
tr_key = []
for i in corpus:
    tr_key.append(keywords.keywords(i, language='english', additional_stopwords=stop_w).split())

# Применение морфологических шаблонов

Отфильтруем выделенные автоматически ключевые слова с помощью добавления составленных паттернов для ключевых слов в эталонной разметке

In [15]:
from nltk import pos_tag

In [16]:
def add_morph(keywords):
    patterns = ['NN ', 'NNS ', 'JJ NNS', 'JJ NN',
               'NNP MD VB NN ', 'NNP NNP NNP ', 'DT']
    pos_tags = []
    for i in keywords:
        res = ''
        out = []
        for j in nltk.pos_tag(i):
            res += (j[1])
            res += ' '
           
            if res in patterns:
                out.append(j[0])
        pos_tags.append(out)
    return pos_tags

In [17]:
m_rake = add_morph(rake_key)
m_tr = add_morph(tr_key)
m_tf = add_morph(tf_key)

# Метрики

Оценим точность, полноту, F-меру выбранных методов относительно эталона: с учётом морфосинтаксических шаблонов и без них

In [18]:
mlb = MultiLabelBinarizer()
def compute_scores(keywords):
    keywords = mlb.transform(keywords)
    print(f"F-score: {f1_score(alls, keywords, average='micro')}")
    print(f"Precision score: {precision_score(alls, keywords, average='micro')}")
    print(f"Recall score: {recall_score(alls, keywords, average='micro')}")


In [50]:
m_k = []
for i in df['my_keywords'].tolist() :
    m_k.append(i.split(', '))

In [53]:
all_tags = m_k + rake_key + tf_key  + tr_key
mlb.fit(all_tags)
alls = mlb.transform(m_k)

In [55]:
print('Rake')
compute_scores(rake_key)
print('')
print('Rake with POS')
compute_scores(m_rake)
print('- - - - - - - - - - - - - - - - - - - - - - - - - -')
print('Tf_idf')
compute_scores(tf_key)
print('')
print('Tf_idf with POS')
compute_scores(m_tf)
print('- - - - - - - - - - - - - - - - - - - - - - - - - -')
print('TextRank')
compute_scores(tr_key)
print('')
print('TextRank with POS')
compute_scores(m_tr)


Rake
F-score: 0.024096385542168672
Precision score: 0.012531328320802004
Recall score: 0.3125

Rake with POS
F-score: 0.0
Precision score: 0.0
Recall score: 0.0
- - - - - - - - - - - - - - - - - - - - - - - - - -
Tf_idf
F-score: 0.04347826086956522
Precision score: 0.03333333333333333
Recall score: 0.0625

Tf_idf with POS
F-score: 0.0
Precision score: 0.0
Recall score: 0.0
- - - - - - - - - - - - - - - - - - - - - - - - - -
TextRank
F-score: 0.012684989429175475
Precision score: 0.0064516129032258064
Recall score: 0.375

TextRank with POS
F-score: 0.10526315789473684
Precision score: 0.3333333333333333
Recall score: 0.0625


/Users/mariadolgodvorova/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Анализ ошибок автоматического выделения ключевых слов

Лучше всего с задачей справился Text-Rank с Pos-тегами, а добавление Pos-тегов в остальных случаях только ухудшило результаты

**Анализ результатов**

- Во-первых, низкие метрики качества связаны с тем, что методы выделяют намного больше слов, чем в эталонной разметке
- Во-вторых, сами корпус - распознанные PDF, и при этом не очищенные

TextRank с Pos-тегами показал лучшие результаты за счет фильтрации лишних слов 